In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pickle import dump
from sklearn import preprocessing

In [ ]:
offset_limit = [[-100, 100], # maju
                [0,100], # maju kanan blokir
                [-100,-1], # maju kiri blokir

                [0,100], # diagonal kanan
                [-100,-1], # diagonal kiri

                [0,100], # kanan
                [-100,-1], # kiri
                [-100, 100], # kanan abs
                [-100, 100], # kiri abs

                [-100, 100], # diem
               ]

# sensor kiri
sensor1_limit = [[10, 200], # maju
                 [10, 200], # maju kanan blockir
                 [10, 50], # maju kiri blockir

                 [10, 200], # diagonal kanan
                 [51, 200], # diagonal kiri

                 [10, 200], # kanan
                 [51, 200], # kiri
                 [10, 50], # kanan abs
                 [51, 200], # kiri abs

                 [10, 50], # diem
                ]

# sensor depan
sensor2_limit = [[101, 200], # maju
                 [51, 100], # maju
                 [51, 100], # maju

                 [51, 100], # diagonal kanan
                 [51, 100], # diagonal kiri

                 [10, 50], # kanan
                 [10, 50], # kiri
                 [10, 50], # kanan abs
                 [10, 50], # kiri abs

                 [10, 50], # diem
                ]

# sensor kanan
sensor3_limit = [[10, 200], # maju
                 [10, 50], # maju
                 [10, 200], # maju

                 [51, 200], # diagonal kanan
                 [10, 200], # diagonal kiri

                 [51, 200], # kanan
                 [10, 200], # kiri
                 [51, 200], # kanan abs
                 [10, 50], # kiri abs 

                 [10, 50], # diem 
                ]

# label
label = ['Maju', 'Maju', 'Maju', 'Diagonal Kanan', 'Diagonal Kiri', 'Kanan', 'Kiri', 'Kanan', 'Kiri', 'Diam']

raw_data_len = 20000
fix_data_len = 10500

In [ ]:
dataset = pd.DataFrame(columns=['offset', 'sensor1','sensor2','sensor3','label'])
for i in range(len(offset_limit)):
    temp_df = pd.DataFrame(columns=['offset', 'sensor1','sensor2','sensor3','label'])
    temp_df['offset'] = np.random.randint(offset_limit[i][0], offset_limit[i][1], raw_data_len)
    temp_df['sensor1'] = np.random.randint(sensor1_limit[i][0], sensor1_limit[i][1], raw_data_len)
    temp_df['sensor2'] = np.random.randint(sensor2_limit[i][0], sensor2_limit[i][1], raw_data_len)
    temp_df['sensor3'] = np.random.randint(sensor3_limit[i][0], sensor3_limit[i][1], raw_data_len)
    temp_df['label'] = label[i]
    temp_df.drop_duplicates(inplace=True)
    dataset = pd.concat([dataset, temp_df.sample(fix_data_len)]).reset_index(drop=True)

In [ ]:
dataset

,offset,sensor1,sensor2,sensor3,label
0,-70,101,113,118,Maju
1,16,151,114,147,Maju
2,15,156,171,155,Maju
3,-17,19,170,12,Maju
4,-25,47,110,78,Maju
...,...,...,...,...,...
104995,-50,33,39,34,Diam
104996,-9,47,17,37,Diam
104997,93,33,43,35,Diam
104998,14,34,10,23,Diam


In [ ]:
x = dataset.iloc[:,0:4].values
y = dataset.iloc[:,4].values

x = np.asarray(x).astype('float32')

In [ ]:
robust_scaler = preprocessing.RobustScaler()
x = robust_scaler.fit_transform(x)

In [ ]:
encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)

In [ ]:
Y = pd.get_dummies(y1).values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, Y, test_size=5000, random_state=0)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.999):
            print("\nAccuracy is Optimal")
            self.model.stop_training = True

callbacks = myCallback()

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(8, input_shape=(4,), activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy'])

model.fit(X_train, y_train, batch_size = 100, epochs=100, callbacks=[callbacks])

Epoch 1/100
1000/1000 [==============================] - 6s 3ms/step - loss: 1.1654 - accuracy: 0.5942
Epoch 2/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.5419 - accuracy: 0.8112
Epoch 3/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3861 - accuracy: 0.8672
Epoch 4/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3297 - accuracy: 0.8830
Epoch 5/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2954 - accuracy: 0.8931
Epoch 6/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2676 - accuracy: 0.9034
Epoch 7/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2441 - accuracy: 0.9131
Epoch 8/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2244 - accuracy: 0.9208
Epoch 9/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2073 - accuracy: 0.9266
Epoch 10/100
1000/1000 [==============================] - 3s 3ms/step - l

In [ ]:
model.evaluate(X_test, y_test)

157/157 [==============================] - 1s 3ms/step - loss: 0.0309 - accuracy: 0.9912


[0.030890537425875664, 0.9911999702453613]

In [ ]:
# save model
model.save('ratna_model.h5')
print('Model Saved!')

# save scaler
dump(robust_scaler, open('scaler.pkl', 'wb'))
print('Scaler Saved!')

Model Saved!
